In [ ]:
# default_exp analyze_by_person

# Analysis By Person

> Module processes actiwatch data for a single individual, with the ability to add additional astral (sunset/sunrise) and sleep timing data.

In [ ]:
#hide
from nbdev.showdoc import *

%run load_actiwatch_data.py
%run firsttime.py

import numpy as np
import pandas as pd

from joblib import *
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

## Loading Actiwatch Data

To begin processing raw data for analysis, all folders containing Actiwatch data should be loaded into a dictionary. Dictionary keys should be identifying names for the folder where the data is stored, which should be used as the value.

For example:

The below directory uses

1. **key** = v1
        v1 indicates visit number 1, or baseline
        
2. **value** = data
        data indicates that the csv files to be loaded are stored in a folder called "data"

In [ ]:
directory = {
    'v1': 'data'
}

In [ ]:
#export
def get_raw_data(key:str, directory: dict, grouping:str = 'Group'):
    """Loads raw actiwatch data for a particular season.
    
        Parameters
        
        key: str

            The key to load actiwatch data from
            For example: v1
            
        directory: dict

            Dictionary of valid seasons to retrieve actiwatch data from
            
        grouping: str

            The name of the generated column for specifying Grouping, where
            values will be the name of the key given. Defaults to 'Group'
            For example: Group
            
        Returns
        
            the loaded raw data for a particular season
    """
    raw_data, summary_data = load_actiwatch_data(directory[key],uidprefix=key)
    raw_data['Group'] = key
    return raw_data

In [ ]:
show_doc(get_raw_data)

<h4 id="get_raw_data" class="doc_header"><code>get_raw_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_raw_data</code>(**`key`**:`str`, **`directory`**:`dict`, **`grouping`**:`str`=*`'Group'`*)

Loads raw actiwatch data for a particular season.

Parameters

key: str

    The key to load actiwatch data from
    For example: v1
    
directory: dict

    Dictionary of valid seasons to retrieve actiwatch data from
    
grouping: str

    The name of the generated column for specifying Grouping, where
    values will be the name of the key given. Defaults to 'Group'
    For example: Group
    
Returns

raw_data

    the loaded raw data for a particular season

An example of output for this function would be:

In [ ]:
raw_data = get_raw_data('v1', directory)
raw_data.head() 

Found 1 csv files in data/. Pass #1, raw data
.
.
Pass #2, data summary
.
.EOF without retrieving summary data: data\v1_sample.csv


,Off-Wrist Status,Activity,Marker,White Light,Red Light,Green Light,Blue Light,Sleep/Wake,Interval Status,UID,Group
DateTime,,,,,,,,,,,
2018-05-07 11:28:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\v1,v1
2018-05-07 11:28:30,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\v1,v1
2018-05-07 11:29:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\v1,v1
2018-05-07 11:29:30,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\v1,v1
2018-05-07 11:30:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\v1,v1


## Recalculating Timing Data (if Necessary)

Sometimes it may be valuable to calculate (or recalculate) the timing data. The ability to do so is provided using the below function, which sets up the timing and raw dataframes. 

In this example, we'll go ahead and use the Seattle as our location for determining sunlight (sunset and sunrise timings), and ask the function to recalculate both the raw and timing data. Example thresholds for lux are also specified below. The outfile specified is where the data will be written to.

In [ ]:
recalculate_raw = False # true forces long recalculations, false loads processed data from disk
recalculate_timing = False
location = 'seattle'
thresholds = [ [5], [10], [50], [100], [500], [1000] ] 
outfile = '../SALA/example_output/'

In [ ]:
#export
def process_timing_data(location: str,
                     outfile: str,
                     thresholds: list,
                     key: str,
                     directory: dict,
                     recalc_raw: bool = False,
                     recalc_timing: bool = False
                     ):
    """Setup timing and raw dataframes or recalculate their
    values if specified and necessary. Both operations take
    a long time and lots of memory. Ill-advised to recalculate
    timing specifically if the data has already been created.
    
    Parameters
    
    location: str
    
        Location for calculating light
        
        For example: seattle
        
    outfile: str
    
        File for re-written data to be placed in, or for data to be loaded from
        
    thresholds: list
    
        List of light thresholds for the watch data
        
    key: str
    
        The key to load actiwatch data from
        
    directory: dict
    
        Dictionary of valid seasons to retrieve actiwatch data from
        
    recalc_raw: bool
    
        Forces long recalculation process if true, loads processed data from disk otherwise
    
    recalc_timing: bool
    
        Forces long recalculation of light timing data, loads it from disk otherwise

    Returns

        (as a tuple) all the data, the timing data for a particular location
    """
    if recalc_raw:
        print("Loading raw data from disk...")
        results = Parallel(n_jobs=len(directory))(delayed(get_raw_data)(key, directory) for key in directory.keys())
        all_data = pd.concat(results)
        all_data.to_parquet(outfile + "raw.parquet", engine = 'fastparquet',
                           compression = "gzip")
    else:
        all_data = pd.read_parquet(outfile + "raw.parquet")
        
    if recalc_timing:
        print("Calculating light timing data...")
        try: results
        except NameError:
            results = (Parallel(n_jobs=len(thresholds))
                (delayed(firstAndLastLight)(all_data, threshold) for threshold in thresholds)
                      )
        timing_data = pd.concat(results)
        
        print("Adding holiday markers to timing data...")
        cal = calendar()
        holidays = (
            cal.holidays(start = timing_data.Date.min(), end = timing_data.Date.max())
        )
        
        nn = pd.DatetimeIndex( timing_data.Date )
        timing_data["DayofWeek"] = nn.dayofweek
        days = ["Mon", "Tues", "Wed", "Thu", "Fri", "Sat", "Sun"]
        day_type = ["Weekday","Weekday","Weekday",
                    "Weekday","Weekday","Weekend/Holiday","Weekend/Holiday"]
        
        day_group = []
        dtp_group = []
        wknd_holiday = []
        
        for index, row in timing_data.iterrows():
            day_group.append(row["Group"].split(location)[0] + days[row["DayofWeek"]])
            if holidays.isin([row["Date"]]).any():
                dtp_group.append(row['Group'].split(location)[0] + "Weekend/Holiday")
                wknd_holiday.append(True)
            else:
                dtp_group.append(
                    row["Group"].split(location)[0] + day_type[row["DayofWeek"]]
                )
                wknd_holiday.append(row['DayofWeek'] > 4)
                
        timing_data["GroupDayofWeek"] = day_group
        timing_data["GroupDayType"] = dtp_group
        timing_data["Weekend/Holiday"] = wknd_holiday
        
        timing_copy = timing_data.copy()
        timing_copy.Date = timing_copy.Date.values.astype("datetime64[s]")
        timing_copy["Watch period"] = pd.to_timedelta(timing_copy["Watch period"])
        timing_copy.to_parquet(
            outfile + "timing.parquet", engine = "fastparquet", compression = "gzip"
        )
    else:
        timing_copy = pd.read_parquet(outfile + "timing.parquet", engine = "fastparquet")
        # return date to original format
        timing_copy.Date = timing_copy.Date.apply(lambda x: x.date())
        timing_data = timing_copy.copy()
    return all_data, timing_data

In [ ]:
show_doc(process_timing_data)

<h4 id="process_timing_data" class="doc_header"><code>process_timing_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>process_timing_data</code>(**`location`**:`str`, **`outfile`**:`str`, **`thresholds`**:`list`, **`key`**:`str`, **`directory`**:`dict`, **`recalc_raw`**:`bool`=*`False`*, **`recalc_timing`**:`bool`=*`False`*)

Setup timing and raw dataframes or recalculate their
values if specified and necessary. Both operations take
a long time and lots of memory. Ill-advised to recalculate
timing specifically if the data has already been created.

Parameters

location: str

    Location for calculating light
    
    For example: seattle
outfile: str

    File for re-written data to be placed in, or for data to be loaded from
    thresholds: list
    
    Should be a list of list of light thresholds for the watch data     
key: str

   The key to load actiwatch data from   
   
directory: dict

    Dictionary of valid seasons to retrieve actiwatch data from
    
recalc_raw: bool

    Forces long recalculation process if true, loads processed data from disk otherwise

recalc_timing: bool

    Forces long recalculation of light timing data, loads it from disk otherwise

Returns

all_data

    all the data
    
timing_data

    the timing data for a particular location

An example of output for this function would be:

In [ ]:
all_data, timing_data = process_timing_data(location, outfile, thresholds, 'v1', directory, False, False)

In [ ]:
all_data.head()

,Off-Wrist Status,Activity,Marker,White Light,Red Light,Green Light,Blue Light,Sleep/Wake,Interval Status,UID,Group
DateTime,,,,,,,,,,,
2018-05-07 11:28:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:28:30,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:29:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:29:30,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1
2018-05-07 11:30:00,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,ACTIVE,v1data\Hero,v1


In [ ]:
timing_data.head()

,UID,Date,Threshold,Last Light,Mins to LL from 4AM,First Light,Mins to FL from 4AM,Time above threshold,Time above threshold AM,Minutes above threshold,Minutes above threshold AM,Lux minutes,Lux minutes AM,Group,Watch period,DayofWeek,GroupDayofWeek,GroupDayType,Weekend/Holiday
index,,,,,,,,,,,,,,,,,,,
0,v1data\Hero,2018-05-17,5,2018-05-17 21:23:30,1043.0,2018-05-17 06:19:30,139.0,0 days 10:16:30,0 days 04:52:00,616.5,292.0,431942.250,266478.420,v1,0 days 00:00:30,3,v1Thu,v1Weekday,False
1,v1data\Hero,2018-05-11,5,2018-05-11 21:52:30,1072.0,2018-05-11 06:15:30,135.0,0 days 10:17:00,0 days 05:04:30,617.0,304.5,706720.190,354358.550,v1,0 days 00:00:30,4,v1Fri,v1Weekday,False
2,v1data\Hero,2018-05-13,5,2018-05-13 21:33:00,1053.0,2018-05-13 06:26:30,146.0,0 days 13:16:30,0 days 05:03:30,796.5,303.5,775243.645,166308.070,v1,0 days 00:00:30,6,v1Sun,v1Weekend/Holiday,True
3,v1data\Hero,2018-05-21,5,2018-05-21 22:39:30,1119.0,2018-05-21 05:29:30,89.0,0 days 12:43:30,0 days 05:39:00,763.5,339.0,408969.375,195590.445,v1,0 days 00:00:30,0,v1Mon,v1Weekday,False
4,v1data\Hero,2018-05-14,5,2018-05-15 01:09:30,1269.0,2018-05-14 05:27:00,87.0,0 days 08:35:30,0 days 03:49:30,515.5,229.5,174880.510,87783.270,v1,0 days 00:00:30,0,v1Mon,v1Weekday,False


## Setting Sunset and Sunrise Timings

Sunset and sunrise timings can also be calculated for actiwatch data. To do so, the specific location to be calculated for is required. Of most importance for the calculation is the longitude and latitude. 

In [ ]:
#hide
from astral import LocationInfo, sun

In [ ]:
#export
def set_sun_timings(timing_data,
                    loc:str,
                    region: str,
                    timezone: str,
                    longitude: float,
                    latitude: float,
                   ):
    """Given a location (city), calculate sunset and sunrise timings for the data
    
    Parameters

    timing_data: pd.DataFrame
    
        Timing data
        
    loc: str (any string)
    
        Name of location to lookup for sunrise/sunset calculations
        
    region: str (any string)
    
        Name of the region the location belongs to
        
    timezone: str  
    
        the location's timezone (a list of time zone names can be obtained from pytz.all_timezones)
    
    longitude: float
    
        Longitude position of the location for sunrise/sunset calculations
        
    latitude: float
    
        Latitude position of the location for sunrise/sunset calculations
    
    Returns
    
        Modified timing data with sunrise and sunset calculations
        
    """ 
    city = LocationInfo(loc, region, timezone, longitude, latitude) 
   
    timing_data["Sunrise"] = timing_data.Date.apply( lambda x: sun.sunrise(city.observer, x))
    timing_data["Sunset"] = timing_data.Date.apply( lambda x: sun.sunset(city.observer, x))
    
    return timing_data
    

In [ ]:
show_doc(set_sun_timings)

<h4 id="set_sun_timings" class="doc_header"><code>set_sun_timings</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>set_sun_timings</code>(**`timing_data`**, **`loc`**:`str`, **`region`**:`str`, **`timezone`**:`str`, **`longitude`**:`float`, **`latitude`**:`float`)

Given a location (city), calculate sunset and sunrise timings for the data

Parameters

timing_data: pd.DataFrame

    Timing data
    
loc: str (any string)

    Name of location to lookup for sunrise/sunset calculations
    
region: str (any string)

    Name of the region the location belongs to
    
timezone: str  

    the location's timezone (a list of time zone names can be obtained from pytz.all_timezones)

longitude: float

    Longitude position of the location for sunrise/sunset calculations
    
latitude: float

    Latitude position of the location for sunrise/sunset calculations

Returns

    Modified timing data with sunrise and sunset calculations
    

An example of output for this function would be:

In [ ]:
timing_data = (
    set_sun_timings(timing_data, "Seattle", "United States", "America/Los_Angeles", 47.2451, 122.438)
)

## Adding Sleep Information

Adding sleep information to the timing data is also possible. The below function adds sleep data, splitting a sleep day at 6pm (under the assumption that people generally do not sleep/end their day at 6pm).

In [ ]:
#export
def process_sleep_data(timing_data, num_sleeps: int = 2):
    """Processes sleep data for existing timing data.
    
    Parameters

    timing_data: pd.DataFrame
    
        Timing data
        
    num_sleeps: int
    
        Cutoff for number of sleeps to display in first resulting frame.
        Default = 2, frame will store days with 3+ sleep instances
    
    Returns
    
        short_frame: pd.DataFrame
        
            Onset, offset, and duration for sleep periods on days with
            more than num_sleeps number of sleep periods
            
        timing_data: pd.DataFrame
        
            Modified timing data with included sleep information
    
    """
    sleepers = []
    sleep_onsets = []
    sleep_offsets = []
    sleep_durations = []
    sleep_onsetMSLMs = []
    sleep_offsetMSLMs = []
    for arow in timing_data.itertuples():    
        UID = arow.UID
        DT = pd.to_datetime(arow.Date)
        TM = pd.to_datetime(DT + pd.Timedelta("1 day"))
        today = DT.strftime("%Y-%m-%d")

        nextday = TM.strftime("%Y-%m-%d")

        # taking raw timing data entry and splitting a "sleep day" at 6pm
        # under the assumption that people do not end their days that early
        day_split = all_data.query("UID == @UID").loc[today +" 18:00":nextday + " 18:00"]

        # REST-S = watch thinks user is asleep
        asleep = day_split[ day_split["Interval Status"] == "REST-S"].copy()

        # there may be more than one sleep period in a given day's data
        # new sleep period = when there is more than 1 hour between successive REST-S entries
        sleep_periods = []
        per = 0
        count = 0

        try:
            lt = asleep.index[0]
            for time in asleep.index:
                # allow up to 1 hour of being awake in the middle of the night
                if (time - lt > pd.Timedelta("1 hour")):
                    per += 1
                lt = time
                sleep_periods.append(per)
            asleep["Sleep period"] = sleep_periods
        except IndexError:
            asleep["Sleep period"] = [pd.to_datetime(0)]


        try:
        # calc sleep onsets/offsets/duration for each period of sleep in a person-day of data
            sleeps = asleep.reset_index().groupby("Sleep period").apply( lambda x: pd.DataFrame({
                     "Sleep onset": [x.DateTime.min()],
                     "Sleep offset": [x.DateTime.max()],
                     "Sleep duration": [x.DateTime.max() - x.DateTime.min()]
                     }, index = x.DateTime.dt.normalize() ))
        except AttributeError:
            sleeps = asleep.reset_index().groupby("Sleep period").apply( lambda x: pd.DataFrame({
             "Sleep onset": [x.DateTime.min()],
             "Sleep offset": [x.DateTime.max()],
             "Sleep duration": [x.DateTime.max() - x.DateTime.min()]
             }))
        sleeps = sleeps.drop_duplicates().sort_values(by="Sleep duration", ascending = False)
        onset = sleeps.iloc[0]['Sleep onset']
        offset = sleeps.iloc[0]['Sleep offset']
        dur =  sleeps.iloc[0]['Sleep duration']
        
        # if onset is actually a datetime
        if not isinstance(onset, np.int64):
            onMSLM = (onset - DT).total_seconds()/60.
        else:
            onMSLM = 0
        
        # if offset is actually a datetime
        if not isinstance(offset, np.int64):
            offMSLM = (offset - TM).total_seconds()/60.
        else:
            offMSLM = 0

        sleep_onsets.append(onset)
        sleep_offsets.append(offset)
        sleep_durations.append(dur)
        sleep_onsetMSLMs.append(onMSLM)
        sleep_offsetMSLMs.append(offMSLM)
        sleep_count = sleeps.shape[0]
        
        # adding to short_frame
        if sleep_count > num_sleeps:
            sleeps['UID']=UID
            sleeps['DT']=DT
            sleeps.reset_index(drop=True).set_index(['UID','DT'])
            sleepers.append(sleeps)
    short_frame = (
                   pd.concat(sleepers).reset_index().drop('DateTime',axis=1)
                   .set_index(['UID','DT']).drop_duplicates()
                   )
    timing_data["Sleep onset"] = sleep_onsets
    timing_data["Sleep offset"] = sleep_offsets
    timing_data["Sleep duration"] = sleep_durations
    timing_data["Sleep onset MSLM"] = sleep_onsetMSLMs
    timing_data["Sleep offset MSLM"] = sleep_offsetMSLMs
    
    return short_frame, timing_data

In [ ]:
show_doc(process_sleep_data)

<h4 id="process_sleep_data" class="doc_header"><code>process_sleep_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>process_sleep_data</code>(**`timing_data`**, **`num_sleeps`**:`int`=*`2`*)

Processes sleep data for existing timing data.

Parameters

timing_data: pd.DataFrame

    Timing data
    
num_sleeps: int

    Cutoff for number of sleeps to display in first resulting frame.
    Default = 2, frame will store days with 3+ sleep instances

Returns

    short_frame: pd.DataFrame
    
        Onset, offset, and duration for sleep periods on days with
        more than num_sleeps number of sleep periods
        
    timing_data: pd.DataFrame
    
        Modified timing data with included sleep information

In [ ]:
short_frame = process_sleep_data(timing_data)[0]
short_frame.head()

Sleep period         Sleep onset        Sleep offset  \
UID         DT                                                                 
v1data\Hero 2018-05-25             1 2018-05-26 00:56:30 2018-05-26 06:37:30   
            2018-05-25             2 2018-05-26 15:19:00 2018-05-26 17:30:30   
            2018-05-25             0 2018-05-25 21:43:00 2018-05-25 23:33:30   
            2018-05-19             0 2018-05-19 21:54:30 2018-05-20 04:59:00   
            2018-05-19             1 2018-05-20 13:10:00 2018-05-20 13:50:30   

                        Sleep duration  
UID         DT                          
v1data\Hero 2018-05-25 0 days 05:41:00  
            2018-05-25 0 days 02:11:30  
            2018-05-25 0 days 01:50:30  
            2018-05-19 0 days 07:04:30  
            2018-05-19 0 days 00:40:30

An example slice of the added sections to the timing data include:

In [ ]:
timing_data = process_sleep_data(timing_data)[1]
timing_data[
    ["Sleep onset", "Sleep offset",
     "Sleep duration", "Sleep onset MSLM",
     "Sleep offset MSLM"]
    ].head()

,Sleep onset,Sleep offset,Sleep duration,Sleep onset MSLM,Sleep offset MSLM
index,,,,,
0,2018-05-17 21:50:00,2018-05-18 05:31:00,0 days 07:41:00,1310.0,331.0
1,2018-05-11 21:58:30,2018-05-12 05:05:00,0 days 07:06:30,1318.5,305.0
2,2018-05-13 21:39:30,2018-05-14 05:16:30,0 days 07:37:00,1299.5,316.5
3,2018-05-21 22:50:00,2018-05-22 06:00:30,0 days 07:10:30,1370.0,360.5
4,2018-05-14 21:37:30,2018-05-15 06:18:00,0 days 08:40:30,1297.5,378.0
